# 🚀 Construction d'un Portefeuille Intelligent (Eightcap + GMV/CVaR)

Ce notebook implémente un pipeline complet de sélection et d'optimisation de portefeuille avec :
- Actions disponibles sur Eightcap (via MetaTrader5)
- Filtrage fondamental SMB et HML
- Estimation des rendements via Fama-French 3 facteurs
- Choix dynamique entre optimisation GMV ou CVaR
- (Exécution optionnelle sur MT5)

In [5]:
# 📦 Chargement des bibliothèques nécessaires
import MetaTrader5 as mt5
import yfinance as yf
import requests
import pandas as pd
import numpy as np
import time
import cvxpy as cp
from tqdm import tqdm
from statsmodels.api import OLS, add_constant

## 🎯 Étape 1 : Tickers Eightcap US uniquement

In [6]:
mt5.initialize()

data = [(s.path.split('\\')[0], s.path.split('\\')[1], s.path.split('\\')[2]) for s in mt5.symbols_get()]

df = pd.DataFrame(data, columns=['type','class','name'])

df.sort_values(['type','class'], ascending=[True,True], inplace=True)

tickers = list( dict.fromkeys(list(df[(df['type']=='Stock') & ((df['class']=='NYS') | (df['class']=='NAS'))]['name'].values)))

mt5.shutdown()

print(df[(df['type']=='Stock') & ((df['class']=='NYS') | (df['class']=='NAS'))]['name'].values)

print(f"{len(tickers)} tickers US récupérés.")

['AAPL' 'ADBE' 'AMAT' 'AMD' 'AMGN' 'AMZN' 'ASML' 'AVGO' 'BNTX' 'CHTR'
 'CMCSA' 'COST' 'CSCO' 'GOOGL' 'HON' 'HOOD' 'INTC' 'INTU' 'ISRG' 'MRNA'
 'MSFT' 'MU' 'NFLX' 'NVDA' 'PDD' 'PEP' 'PYPL' 'QCOM' 'SBUX' 'TMUS' 'TSLA'
 'TXN' 'ZM' 'SHOP' 'ROKU' 'ADSK' 'AEP' 'ALGN' 'ANSS' 'BIDU' 'BIIB' 'BKNG'
 'CDNS' 'CDW' 'CHKP' 'CPRT' 'CRWD' 'CSX' 'CTAS' 'CTSH' 'DLTR' 'DOCU' 'EA'
 'EBAY' 'EXC' 'FAST' 'FOX' 'GILD' 'IDXX' 'ILMN' 'INCY' 'KHC' 'KLAC' 'LRCX'
 'LULU' 'MAR' 'MCHP' 'MDLZ' 'MELI' 'MNST' 'MRVL' 'NTES' 'NXPI' 'ORLY'
 'PAYX' 'PCAR' 'REGN' 'ROST' 'SIRI' 'SNPS' 'SWKS' 'VRSK' 'VRSN' 'VRTX'
 'WBA' 'WDAY' 'XEL' 'ADI' 'JD_US' 'MTCH' 'TCOM' 'META' 'PLTR' 'GEHC' 'WBD'
 'ADP' 'APD' 'CME' 'EQIX' 'COIN' 'CEG' 'ABBV' 'ABT' 'ACN' 'BAC' 'BBY'
 'BHP_US' 'BMY' 'BRK.A' 'CAT' 'CRM' 'CVX' 'DAL' 'DHR' 'DIS' 'F' 'FCX' 'GE'
 'GM' 'GS' 'HD' 'JNJ' 'JPM' 'KO' 'LUV' 'MA' 'MCD' 'MDT' 'MRK' 'NEE' 'NKE'
 'NVO' 'NVS' 'ORCL' 'PFE' 'PG' 'PHM' 'PM' 'SAP_US' 'T' 'TM' 'TMO' 'TSM'
 'TSN' 'UNH' 'USB' 'V' 'VZ' 'WFC' 'WMT' 'XOM' 'AR' 'FD

## 📑 Étape 2 : Filtrage fondamental SMB / HML avec API FMP

### Parametrages API

In [ ]:
API_KEY = "<YOUR_API_KEY>"


### Construire SMB et HML

In [ ]:
fundamentals = {}

for t in tqdm(tickers, desc="Téléchargement des données fondamentales"):
    base = t.replace('.US', '')

    # Profil : Market Cap & Price
    url = f"https://financialmodelingprep.com/api/v3/profile/{base}?apikey={API_KEY}"
    try:
        r = requests.get(url)
        if r.ok and r.json():
            d = r.json()[0]
            fundamentals[t] = {
                "marketCap": d.get("mktCap"),
                "price": d.get("price")
            }
    except:
        continue
    time.sleep(1.2)

    # Ratios : Price-to-Book
    url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{base}?apikey={API_KEY}"
    try:
        r = requests.get(url)
        if r.ok and r.json():
            d = r.json()[0]
            pb = d.get("priceToBookRatioTTM")
            if pb and pb > 0:
                fundamentals[t]["bookValue"] = fundamentals[t]["price"] / pb
    except:
        continue
    time.sleep(1.2)    

df = pd.DataFrame(fundamentals).T.dropna()

if "bookValue" in df.columns:
    df["B/M"] = 1 / df["bookValue"]
else:
    raise KeyError("La colonne 'bookValue' est absente du DataFrame.")
df["Size_rank"] = df["marketCap"].rank()
df["BM_rank"] = df["B/M"].rank(ascending=False)

print(df) 

small_cut = df["Size_rank"].quantile(0.3)
value_cut = df["BM_rank"].quantile(0.3)
 

filtered = df[(df["Size_rank"] <= small_cut) & (df["BM_rank"] <= value_cut)]
filtered_tickers = [t.replace('.US', '') for t in filtered.index]
print(f"{len(filtered_tickers)} tickers sélectionnés selon SMB / HML.")

Téléchargement des données fondamentales: 100%|██████████| 202/202 [10:07<00:00,  3.01s/it]

         marketCap     price   bookValue       B/M  Size_rank  BM_rank
AAPL  3.132336e+12  209.7200    4.453126  0.224561      181.0      9.0
ADBE  1.531829e+11  361.1100   26.543368  0.037674      117.0     91.0
AMAT  1.542708e+11  192.2382   23.425254  0.042689      119.0     83.0
AMD   2.504252e+11  154.4500   35.738268  0.027981      148.0    117.0
AMGN  1.597632e+11  297.1200   11.554480  0.086547      122.0     38.0
...            ...       ...         ...       ...        ...      ...
SOLV  1.253825e+10   72.4700   18.783393  0.053239        7.0     69.0
AEM   6.063440e+10  119.9400   43.076077  0.023215       62.0    128.0
NEM   6.467643e+10   58.1100   27.751332  0.036034       67.0     97.0
TRV   5.692093e+10  251.2300  124.152737  0.008055       56.0    175.0
B     3.654712e+10   21.2550   14.173913  0.070552       35.0     48.0

[183 rows x 6 columns]
16 tickers sélectionnés selon SMB / HML.


CONSTRUCTION DU MOM(le facteur Momentum)

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# On part de ton DataFrame df qui contient au moins ces colonnes:
# df.index = tickers avec '.US'
# df["marketCap"], df["price"], df["bookValue"], etc.

# Paramètres MOM
lookback_months = 12
exclude_last_month = True
top_quantile = 0.3
bottom_quantile = 0.3

# Initialiser un dict pour stocker MOM
momentum = {}

tickers = df.index.tolist()  # Utiliser les mêmes tickers que dans df (SMB/HML)

for t in tqdm(tickers, desc="Téléchargement des prix historiques pour MOM"):
    ticker_yf = t.replace('.US', '')
    try:
        # Téléchargement 13 mois (12 + 1), auto_adjust=True pour avoir 'Close' ajusté
        df_hist = yf.download(
            ticker_yf, 
            period=f"{lookback_months+1}mo", 
            interval="1d", 
            progress=False, 
            auto_adjust=True
        )
        if df_hist.empty:
            continue
        
        prices = df_hist['Close']  # prix déjà ajustés
        
        month_ends = prices.resample('M').last()
        if len(month_ends) < lookback_months + 1:
            continue
        
        if exclude_last_month:
            start_price = month_ends.iloc[-(lookback_months+1)]
            end_price = month_ends.iloc[-2]
        else:
            start_price = month_ends.iloc[-lookback_months]
            end_price = month_ends.iloc[-1]
        
        ret = (end_price / start_price) - 1
        momentum[t] = ret
    except Exception as e:
        print(f"Erreur sur {t}: {e}")
    time.sleep(1)

# Ajouter la colonne momentum_return dans df
df_mom = pd.DataFrame.from_dict(momentum, orient='index', columns=['momentum_return'])

# Joindre à df initial (SMB/HML)
df = df.join(df_mom)

# Supprimer les lignes sans momentum_return
df = df.dropna(subset=['momentum_return'])

# Classement MOM (rang, meilleur = 1)
df['MOM_rank'] = df['momentum_return'].rank(ascending=False)

# Seuils quantiles MOM
top_threshold = df['MOM_rank'].quantile(top_quantile)
bottom_threshold = df['MOM_rank'].quantile(1 - bottom_quantile)

# Sélection winners/losers
df['MOM_winner'] = df['MOM_rank'] <= top_threshold
df['MOM_loser'] = df['MOM_rank'] >= bottom_threshold

winners = df[df['MOM_winner']]
losers = df[df['MOM_loser']]

print(f"{len(winners)} tickers winners (top {int(top_quantile*100)}%)")
print(f"{len(losers)} tickers losers (bottom {int(bottom_quantile*100)}%)")

# Calcul facteur WML (momentum)
mean_winner_ret = winners['momentum_return'].mean()
mean_loser_ret = losers['momentum_return'].mean()
wml_factor = mean_winner_ret - mean_loser_ret

print(f"Facteur WML (Momentum) = {wml_factor:.4f}")

# Afficher extrait des tickers winners/losers sans '.US'
print("Extrait winners:", [t.replace('.US','') for t in winners.index[:5]])
print("Extrait losers:", [t.replace('.US','') for t in losers.index[:5]])


Téléchargement des prix historiques pour MOM:   0%|          | 0/183 [00:00<?, ?it/s]

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_9012\2675180069.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_ends = prices.resample('M').last()
Téléchargement des prix historiques pour MOM:   1%|          | 1/183 [00:08<25:03,  8.26s/it]C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_9012\2675180069.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_ends = prices.resample('M').last()
Téléchargement des prix historiques pour MOM:   1%|          | 2/183 [00:09<12:31,  4.15s/it]C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_9012\2675180069.py:38: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_ends = prices.resample('M').last()
Téléchargement des prix historiques pour MOM:   2%|▏         | 3/183 [00:10<08:31,  2.84s/it]C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_9012\26751

0 tickers winners (top 30%)
0 tickers losers (bottom 30%)
Facteur WML (Momentum) = nan
Extrait winners: []
Extrait losers: []


In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm

# 1. Télécharger les prix des actions sélectionnées
prices = yf.download(filtered_tickers, start="2023-01-01", end="2025-07-01", auto_adjust=True)["Close"]

# 2. Calcul des rendements quotidiens
returns = prices.pct_change().dropna()
returns = returns.dropna(axis=1)

# 3. Paramètres Momentum
lookback_days = 252  # 12 mois
skip_days = 21       # sauter le dernier mois
top_quantile = 0.3
bottom_quantile = 0.3

momentum_factors = []
dates = prices.index[lookback_days + skip_days:]

for current_date in tqdm(dates, desc="Construction WML"):
    try:
        # Fenêtre pour mesurer le momentum
        end_idx = prices.index.get_loc(current_date) - skip_days
        start_idx = end_idx - lookback_days
        
        if start_idx < 0:
            momentum_factors.append(np.nan)
            continue
        
        start_date = prices.index[start_idx]
        end_date = prices.index[end_idx]
        
        perf = prices.loc[end_date] / prices.loc[start_date] - 1
        
        # Sélection des gagnants/perdants
        top_thresh = perf.quantile(1 - top_quantile)
        bottom_thresh = perf.quantile(bottom_quantile)
        
        winners = perf[perf >= top_thresh].index
        losers = perf[perf <= bottom_thresh].index
        
        # Calcul de la performance quotidienne
        prev_idx = prices.index.get_loc(current_date) - 1
        ret_today = prices.iloc[prev_idx + 1] / prices.iloc[prev_idx] - 1
        
        wml = ret_today[winners].mean() - ret_today[losers].mean()
        momentum_factors.append(wml)
        
    except Exception:
        momentum_factors.append(np.nan)

# 4. Série finale WML
wml_series = pd.Series(momentum_factors, index=dates, name='WML').dropna()
wml_series.index = pd.to_datetime(wml_series.index)


[*********************100%***********************]  16 of 16 completed
Construction WML: 100%|██████████| 351/351 [00:02<00:00, 140.34it/s]


In [11]:
# Charger les facteurs Fama-French 3F + RF
factors = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
factors = factors.rename(columns={'Mkt-RF': 'MKT'})
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y%m%d')
factors = factors.set_index('Date') / 100
factors = factors[['MKT', 'SMB', 'HML', 'RF']]

# Ajouter WML
factors = factors.join(wml_series)

# Alignement avec les rendements
returns, factors = returns.align(factors, join='inner', axis=0)


In [21]:
from statsmodels.api import OLS, add_constant

mu = []

for asset in returns.columns:
    y = returns[asset] - factors['RF']
    X = add_constant(factors[['MKT', 'SMB', 'HML', 'WML']])
    model = OLS(y, X).fit()
    mu.append(model.predict(X.mean()).iloc[0] + factors['RF'].mean())

mu = np.array(mu)
cov = returns.cov()

print("Espérance de rendement (mu):", mu)
print("Matrice de covariance calculée.")



KeyboardInterrupt



## 📈 Étape 3 : Téléchargement des prix et rendements

In [20]:
returns = yf.download(filtered_tickers, start="2024-01-01", end="2025-07-17")["Close"].pct_change().dropna().dropna(axis=1)
returns

[*********************100%***********************]  16 of 16 completed


Ticker,B,BBY,CPRT,DOCU,EBAY,EL,F,FAST,ILMN,MCHP,PAYX,RDDT,VRSK,WBA,WBD,XPEV
Date,,,,,,,,,,,,,,,,
2024-03-22,-0.018343,0.015167,0.003496,-0.007325,-0.011154,-0.020991,-0.000774,-0.004081,-0.001528,-0.009339,-0.012180,-0.088025,-0.004451,-0.010577,-0.032110,-0.077922
2024-03-25,0.003222,-0.011511,-0.004006,0.006178,-0.003695,-0.028007,-0.000775,-0.013956,-0.023171,-0.004430,-0.015558,0.300000,-0.012177,0.002430,0.002370,0.005868
2024-03-26,-0.003211,-0.008176,0.000700,-0.010745,-0.002928,0.001006,-0.035659,-0.002987,-0.001939,-0.016087,0.005212,0.088796,-0.003405,-0.005817,-0.009456,-0.003501
2024-03-27,0.044459,0.022608,0.000350,0.021897,0.016445,0.041131,0.049839,0.006512,0.036472,0.034207,0.016307,-0.113039,0.011764,0.024866,0.031026,-0.085480
2024-03-28,0.026527,0.001954,0.011880,0.004724,0.016564,0.062810,0.016845,-0.001812,-0.009807,0.005830,0.010450,-0.145974,0.007695,0.031874,0.010417,-0.016645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-10,0.008134,0.017389,0.007347,-0.040462,0.011792,0.063388,0.006762,0.003687,0.026786,0.005356,-0.004317,-0.018194,-0.002700,-0.001727,0.013925,-0.012465
2025-07-11,0.007119,-0.026180,-0.011044,-0.036799,-0.001425,-0.011075,-0.010915,-0.006658,-0.007996,-0.006926,-0.013901,0.018112,-0.007890,-0.003460,0.006867,-0.001721
2025-07-14,0.000943,-0.011701,-0.006953,0.036166,0.006744,-0.016359,0.008489,0.041599,-0.007254,-0.006840,0.007118,0.005426,0.017004,-0.000868,0.023870,0.013793


## 📊 Étape 4 : Estimation Fama-French 3

In [14]:
factors = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
factors = factors.rename(columns={'Mkt-RF': 'MKT'})
factors['Date'] = pd.to_datetime(factors['Date'], format='%Y%m%d')
factors = factors.set_index('Date') / 100
factors = factors[['MKT', 'SMB', 'HML', 'RF']]

print(factors)

returns, factors = returns.align(factors, join='inner', axis=0)

mu = []
for asset in returns.columns:
    y = returns[asset] - factors['RF']
    X = add_constant(factors[['MKT', 'SMB', 'HML']])
    model = OLS(y, X).fit()
    mu.append(model.predict(X.mean()).iloc[0] + factors['RF'].mean())


mu = np.array(mu)
cov = returns.cov()

               MKT     SMB     HML       RF
Date                                       
1926-07-01  0.0009 -0.0025 -0.0027  0.00009
1926-07-02  0.0045 -0.0033 -0.0006  0.00009
1926-07-06  0.0017  0.0030 -0.0039  0.00009
1926-07-07  0.0009 -0.0058  0.0002  0.00009
1926-07-08  0.0022 -0.0038  0.0019  0.00009
...            ...     ...     ...      ...
2025-03-25  0.0012 -0.0094  0.0005  0.00016
2025-03-26 -0.0122 -0.0048  0.0152  0.00016
2025-03-27 -0.0042  0.0019  0.0030  0.00016
2025-03-28 -0.0207 -0.0048  0.0040  0.00016
2025-03-31  0.0040 -0.0108  0.0037  0.00016

[25961 rows x 4 columns]


## ⚖️ Étape 5 : Optimisation dynamique GMV ou CVaR

In [15]:
vol_threshold = 0.02
market_vol = factors['MKT'].rolling(20).std().iloc[-1]

if market_vol > vol_threshold:
    print("🔺 Volatilité élevée – CVaR")
    R = returns.values
    w = cp.Variable(R.shape[1])
    z = cp.Variable(R.shape[0])
    VaR = cp.Variable()
    alpha = 0.05
    obj = cp.Minimize(VaR + (1 / (alpha * R.shape[0])) * cp.sum(z))
    constraints = [cp.sum(w) == 1, w >= 0, z >= 0, z >= -R @ w - VaR]
    cp.Problem(obj, constraints).solve()
    weights = w.value
else:
    print("🟢 Volatilité normale – GMV")
    w = cp.Variable(len(mu))
    obj = cp.Minimize(cp.quad_form(w, cov.values))
    constraints = [cp.sum(w) == 1, w >= 0]
    cp.Problem(obj, constraints).solve()
    weights = w.value

🟢 Volatilité normale – GMV


## 🧾 Étape 6 : Envoi automatique des ordres sur Eightcap (via MetaTrader 5)

In [17]:
# 🚀 Étape 6 (optionnelle) : Envoi des ordres sur MetaTrader 5 (Eightcap)
# Assurez-vous que MetaTrader 5 est lancé et connecté au bon compte

capital = 10000  # Capital à investir
mt5.initialize()

for symbol, weight in zip(returns.columns, weights):
    eightcap_symbol = symbol
    info = mt5.symbol_info_tick(eightcap_symbol)
    if info is None:
        print(f"⛔️ {eightcap_symbol} non disponible avec weight={weight}")
        continue
    lot = round((weight * capital / 100) / info.ask, 2)
    entry_price = info.ask

    try:
        ticker_yf = symbol.replace(".US", "")
        df = yf.download(ticker_yf, period="2mo", interval="1d")

        if df.empty or len(df) < 15:
            print(f"⚠️ Données insuffisantes pour {symbol}, on saute.")
            continue

        # Vérifie que ce sont bien des Series (pas des DataFrames multicolonnes)
        high = df['High'].values.flatten()
        low = df['Low'].values.flatten()
        close = df['Close'].values.flatten()

        tr = np.maximum(high - low, np.maximum(abs(high - np.roll(close, 1)), abs(low - np.roll(close, 1))))
        tr[0] = high[0] - low[0]  # pour éviter NaN au début
        atr = pd.Series(tr).rolling(14).mean().iloc[-1]

    except Exception as e:
        print(f"⚠️ Impossible de calculer l’ATR pour {symbol}, erreur : {e}")
        continue

    # Calcul SL et TP dynamiques
    stop_loss = round(entry_price - 1.5 * atr, 2)
    take_profit = round(entry_price + 2.5 * atr, 2)
    print(stop_loss)
    print(take_profit)
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": eightcap_symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": info.ask,
        "sl": stop_loss,
        "tp": take_profit,
        "deviation": 20,
        "magic": 20250523,
        "comment": "Portefeuille Auto CVaR/GMV",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    print(f"✅ {eightcap_symbol}: volume {lot} envoyé ({result.retcode})")

import time

# Attendre quelques secondes (ou minutes) si besoin
time.sleep(10)

# Lire toutes les positions ouvertes
positions = mt5.positions_get()

# Vérifie qu'il y a des positions ouvertes
if positions:
    total_profit = sum(pos.profit for pos in positions)
    print(f"Profit total latent: {total_profit:.2f} €")

    # Vérifie si la perte dépasse 1% du capital
    if total_profit <= -capital * 0.01:
        print("Perte >= 1% du capital. Fermeture de toutes les positions.")

        for pos in positions:
            close_request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": pos.symbol,
                "volume": pos.volume,
                "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                "position": pos.ticket,
                "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                "deviation": 20,
                "magic": 20250523,
                "comment": "Fermeture automatique perte > 1%",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
            result = mt5.order_send(close_request)
            print(f"Fermeture de {pos.symbol}: retcode={result.retcode}")
else:
    print("Aucune position ouverte.")

mt5.shutdown()

[*********************100%***********************]  1 of 1 completed


20.55
22.63
✅ B: volume 0.55 envoyé (10009)


[*********************100%***********************]  1 of 1 completed


63.31
72.35
✅ BBY: volume 0.07 envoyé (10009)


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


11.93
13.46
✅ WBD: volume -0.0 envoyé (10014)


[*********************100%***********************]  1 of 1 completed


16.87
19.08
✅ XPEV: volume 0.08 envoyé (10009)
Profit total latent: -10.31 €


True